In [1]:
# 라이브러리 호출
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bts
import json
import re
import time
from tqdm.notebook import tqdm

In [2]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\2-13\\Desktop\\DMF\\final\\naver'

In [3]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['.ipynb_checkpoints',
 'crawling.py',
 'data',
 'LG에너지솔루션.ipynb',
 'LG에너지솔루션step2.ipynb',
 'naver_news_hinix.csv',
 'Naver_News_List.pkl',
 'Naver_News_List_2.pkl',
 'Naver_News_List_3.pkl',
 'naver_news_samsung.csv',
 'SK하이닉스크롤링.ipynb',
 'SK하이닉스크롤링step2.ipynb',
 '삼전 크롤링 step2.ipynb',
 '삼전크롤링.ipynb']

In [4]:
# pkl 파일을 읽고 newsList 생성
newsList = pd.read_pickle(filepath_or_buffer = 'Naver_News_List_3.pkl')

In [5]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date
289,세계일보,"LG엔솔·삼성SDI, 전기상용차 배터리 선보여",https://n.news.naver.com/mnews/article/022/000...,A15면 1단
290,파이낸셜뉴스,"LG엔솔·삼성SDI ""차세대 배터리 솔루션 선도""",https://n.news.naver.com/mnews/article/014/000...,14면 1단
291,디지털타임스,"이달훈 LG엔솔 상무 ""셀 13만개 분해한 데이터 기반 `BMTS` 개발"" 자신감",https://n.news.naver.com/mnews/article/029/000...,3주 전
292,연합뉴스,"LG엔솔, CATL 제치고 중국 현지서 배터리 품질 부문 1위 차지",https://n.news.naver.com/mnews/article/001/001...,3주 전
293,헤럴드경제,“신재생 에너지가 전기 가장 많이 만드는 시대 온다” [헤럴드 기업포...,https://n.news.naver.com/mnews/article/016/000...,4면 TOP


In [6]:
# newsList의 정보 확인
newsList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   press   294 non-null    object
 1   title   294 non-null    object
 2   nlink   294 non-null    object
 3   date    294 non-null    object
dtypes: object(4)
memory usage: 9.3+ KB


In [7]:
# 네이버뉴스 링크로 HTTP 요청을 실행하고 뉴스 본문을 반환하는 함수 생성
def NaverNewsBody(nlink):
    
    # nlink에 따라 HTTP 요청 실행 및 응답 바디 문자열 처리 코드 분기
    if 'https://n.news' in nlink:
        
        # 네이버뉴스 링크로 HTTP 요청 실행
        res = requests.get(url = nlink)
        
        # HTTP 응답 바디 문자열을 bs4.BeautifulSoup 객체로 변환
        soup = bts(markup = res.text, features = 'html.parser')
        
        # soup에서 뉴스 본문을 포함하는 HTML 요소 선택
        item = soup.select_one(selector = '#dic_area, div._article_content')
        
        # item의 텍스트 반환
        # [참고] strip()은 문자열 양 옆에 있는 공백을 모두 제거함
        return item.text.strip()
    
    elif 'https://m' in nlink:
        
        # 'https://m'으로 시작하는 nlink의 일부 문자열 변경한 url 생성
        url = re.sub(pattern = 'https://m.(sports|entertain).naver.com(.+)?(?=/article)', 
                     repl = r'https://api-gw.\1.naver.com/news', 
                     string = newsList['nlink'].iloc[i])
        
        # url로 HTTP 요청 실행
        res = requests.get(url = url)
        
        # JSON 형태의 문자열을 딕셔너리로 변환
        dic = json.loads(s = res.text)
        
        # 뉴스 본문 반환
        return dic['result']['articleInfo']['article']['content']

In [8]:
# 인덱스 설정: 첫 번째 원소
# [참고] 반복문으로 실행할 코드에 인덱스를 추가하면 편리함
i = 0

In [9]:
# 해당 인덱스의 nlink 확인
print(newsList['nlink'].iloc[i])

https://n.news.naver.com/mnews/article/015/0005044471


In [10]:
# 함수 테스트
NaverNewsBody(nlink = newsList['nlink'].iloc[i])

'LG엔솔, 포드에 13兆 배터리 공급 \'잭팟\'전기차 캐즘에도 잇단 수주109GWh 규모…밴 100만대분지난주 벤츠 이어 대규모 계약폴란드 공장 가동률 대폭 늘 듯상용차용 배터리 생산 까다로워승용차보다 고출력·내구성 필요LG엔솔 "품질·성능 인정받아"\n\n\n\n서울 여의도 LG에너지솔루션 본사. 사진=한경DBLG에너지솔루션이 미국 빅3 완성차 업체인 포드로부터 13조원이 넘는 상용차용 배터리 물량을 수주했다. 지난 8일 메르세데스벤츠와 수조원 규모의 중대형 원통형 배터리 공급 계약을 맺은 지 1주일 만에 다시 한번 ‘잭팟’을 터뜨렸다. LG에너지솔루션이 삼원계 배터리, 리튬·인산철(LFP) 배터리, 에너지저장장치(ESS)용 배터리 등으로 포트폴리오를 다각화한 덕분에 전기차 캐즘(일시적 수요둔화)을 이겨내고 있다는 평가가 나온다.1주일 만에 또 대규모 수주LG에너지솔루션은 포드와 총 109기가와트시(GWh) 규모의 배터리 셀과 모듈을 공급하는 계약을 15일 맺었다고 공시했다. 이 회사가 따낸 전기 상용차용 배터리 계약 가운데 가장 많은 물량이다. LG에너지솔루션은 NCMA(니켈·코발트·망간·알루미늄) 파우치형 배터리를 폴란드 공장에서 생산해 포드에 공급할 계획이다. LG에너지솔루션은 앞서 지난 8일 메르세데스벤츠와 10년간 50.5GWh 규모의 중대형 원통형 배터리 공급 계약을 맺었다.\n\n\n\n이번 계약은 2027~2032년 6년간 75GWh와 2026~2030년 5년간 34GWh를 공급하는 두 건으로 이뤄졌다. 지난해 두 회사가 추진했다가 접은 튀르키예 합작공장 물량에 신규 물량이 추가됐다. LG에너지솔루션은 유럽 전기차 판매 둔화로 가동률이 떨어진 폴란드 공장(연산 90GWh)의 생산성을 높일 수 있게 됐다. 60%로 추정되는 이 공장 가동률은 LG에너지솔루션 실적의 발목을 잡고 있었다. 회사는 폴란드 공장 생산라인을 ESS용, LFP용 등으로 다양화해 생산 효율성을 끌어올릴 계획이다.회사 측은 수주금액을 밝히지 않았지만 업계에서는 13조원 이상

In [11]:
# 경고를 출력하지 않도록 설정
import warnings
warnings.filterwarnings('ignore')

In [12]:
# 반복 실행할 횟수 생성
n = len(newsList)
print(n)

294


In [13]:
# newsList에 뉴스 본문을 저장할 열 추가
newsList['body'] = np.nan

# 반복문으로 네이버뉴스 본문 수집
for i in tqdm(range(n)):
    
    # 네이버뉴스 링크로 뉴스 본문 수집
    newsList['body'].iloc[i] = NaverNewsBody(nlink = newsList['nlink'].iloc[i])
    
    # 1초간 멈춤
    time.sleep(1)

  0%|          | 0/294 [00:00<?, ?it/s]

In [14]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date,body
289,세계일보,"LG엔솔·삼성SDI, 전기상용차 배터리 선보여",https://n.news.naver.com/mnews/article/022/000...,A15면 1단,‘IAA 트랜스포테이션’ 참석차세대 제품·솔루션 등 공개LG에너지솔루션과 삼성SDI...
290,파이낸셜뉴스,"LG엔솔·삼성SDI ""차세대 배터리 솔루션 선도""",https://n.news.naver.com/mnews/article/014/000...,14면 1단,최대 상용차전시회 獨 IAA서 공개 \n\n\n\n오는 22일까지 독일 하노버에서 ...
291,디지털타임스,"이달훈 LG엔솔 상무 ""셀 13만개 분해한 데이터 기반 `BMTS` 개발"" 자신감",https://n.news.naver.com/mnews/article/029/000...,3주 전,LG에너지솔루션이 BMTS(배터리 관리 토탈 솔루션) 사업의 새로운 브랜드인 'B....
292,연합뉴스,"LG엔솔, CATL 제치고 중국 현지서 배터리 품질 부문 1위 차지",https://n.news.naver.com/mnews/article/001/001...,3주 전,(서울=연합뉴스) 장하나 기자 = LG에너지솔루션이 중국 배터리 업체인 CATL과 ...
293,헤럴드경제,“신재생 에너지가 전기 가장 많이 만드는 시대 온다” [헤럴드 기업포...,https://n.news.naver.com/mnews/article/016/000...,4면 TOP,이우현 OCI홀딩스 회장친환경 전환 거스를 수 없는 기류태양광이 지배적 발전방식 될...


In [15]:
# 결과를 CSV 파일로 저장
newsList.to_csv('naver_news_3.csv', index=False, encoding='utf-8-sig')